In [3]:
import numpy as np
import pandas as pd
import os
import pdb
import csv
import math
import matplotlib.pyplot as plta
import pylab
import matplotlib.animation as animation
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
from mpl_toolkits.mplot3d import Axes3D
def distance(p0, p1):
    return math.sqrt((p0[0] - p1[0])**2 + (p0[1] - p1[1])**2 +(p0[2] - p1[2])**2  )
def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy
def midpoint(x1, y1, z1, x2, y2, z2):
    return ((x1 + x2)/2, (y1 + y2)/2, (z1 + z2)/2)


# Open up water peaks 

In [4]:
colspecs_pdb = [(0, 6), (6, 11), (12, 16), (16, 17), (17, 20), (21, 22), (22, 26),
            (26, 27), (30, 38), (38, 46), (46, 54), (54, 60), (60, 66), (76, 78),
            (78, 80)]


names_pdb = ['ATOM', 'serial', 'name', 'altloc', 'resname', 'chainid', 'resseq',
         'icode', 'x', 'y', 'z', 'occupancy', 'tempfactor', 'element', 'charge']

pdb_path = 'files/peaks_SOL_rms2.69_7_13_22.pdb'
peaks = pd.read_fwf(pdb_path, names=names_pdb, colspecs=colspecs_pdb,skiprows=4)
peaks_in_roi = peaks.copy()

# Load up water channel bounds

In [30]:
chain='G'
chan = 'O4'

In [31]:
# read .dat to a list of lists
datContent = [i.strip().split() for i in open("files/%s.dat" %chan).readlines()]

# write it as a new CSV file
with open("./file.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerows(datContent)
names_csv = ['Water', 'Atom_1', 'Res_1',  'Chain_1', 'Resnum_1','comma', 'Atom_2', 'Res_2', 'Chain_2','Resnum_2']
file_csv = pd.read_csv('file.csv', names=names_csv)
file_csv['Water']= file_csv['Water'].str.strip(',')
file_csv['Chain_2']
if chain =='g':
    file_csv['Chain_1']= file_csv['Chain_1'].str.lower()
    file_csv['Chain_2']= file_csv['Chain_2'].str.lower()
file_csv

,Water,Atom_1,Res_1,Chain_1,Resnum_1,comma,Atom_2,Res_2,Chain_2,Resnum_2
0,19,OD1,ASP,A,61,",",NH2,ARG,C,357
1,20,OE2,GLU,C,354,",",CB,ASP,A,61
2,48,O,ASP,A,61,",",ND2,ASN,A,87
3,49,CG2,ILE,A,63,",",C,GLY,C,353
4,50,OG1,THR,C,335,",",CA,GLY,C,353
5,51,OG1,THR,C,335,",",O,LEU,C,337
6,52,O,ASN,A,335,",",ND2,ASN,A,338
7,53,O,GLY,C,338,",",SD,MET,C,342
8,71,O,ASN,A,338,",",OD1,ASN,U,99
9,72,CD,PRO,C,334,",",ND2,ASN,D,350


# Get boundaries along channel (reference_bound_1 and reference_bound_2), and then get dataframe of XTAL waters

In [32]:
gro_path = 'files/0F.pdb_fittedto55all.pdb'
allrois=[]

reference = pd.read_fwf(gro_path, names=names_pdb, colspecs=colspecs_pdb, skiprows=4)
reference_bound_1 = []
reference_bound_2 = []

for i,row in file_csv.iterrows():
    reference_bound_1.append(reference[(reference['resname']==file_csv['Res_1'].iloc[i]) & (reference['name']==file_csv['Atom_1'].iloc[i]) &(reference['chainid']==file_csv['Chain_1'].iloc[i]) & (reference['resseq']==file_csv['Resnum_1'].iloc[i])])
    reference_bound_2.append(reference[(reference['resname']==file_csv['Res_2'].iloc[i]) & (reference['name']==file_csv['Atom_2'].iloc[i]) &(reference['chainid']==file_csv['Chain_2'].iloc[i]) & (reference['resseq']==file_csv['Resnum_2'].iloc[i])])
    roi=3.5
    if ((file_csv['Water'].iloc[i] == '40') or (file_csv['Water'].iloc[i] == '121') or (file_csv['Water'].iloc[i] == '24') or  (file_csv['Water'].iloc[i] == '61')):  
        print('Update ROI for water', file_csv['Water'].iloc[i])
        roi = 5
    allrois.append(roi)
    
ref2=reference[(reference['resname']=='OOO')|(reference['resname']=='HOH')]
df =ref2

## Make list of waters found in the defined channel

In [33]:
ref_waters = []

reference_midpt=pd.DataFrame(columns=['Water','x','y','z'],dtype=object)
reference_midpt['Water'] = file_csv['Water']
for i,rew in file_csv.iterrows():
    mid= midpoint(reference_bound_1[i]['x'].values[0],reference_bound_1[i]['y'].values[0],reference_bound_1[i]['z'].values[0],reference_bound_2[i]['x'].values[0],reference_bound_2[i]['y'].values[0],reference_bound_2[i]['z'].values[0])
    reference_midpt['x'][i] =mid[0]
    reference_midpt['y'][i] =mid[1]
    reference_midpt['z'][i] =mid[2]

for n,rew in df.iterrows(): 
    point1= [float(df.loc[n,'x']),float(df.loc[n,'y']),float(df.loc[n,'z'])]
    for k,pew in  reference_midpt.iterrows():
        channel_roi = allrois[k]
        point2= [float(reference_midpt.loc[k,'x']),float(reference_midpt.loc[k,'y']),float(reference_midpt.loc[k,'z'])]
        if ((distance(point1,point2) < channel_roi )):
            print ("Water ID = %0.2f" %df['resseq'].loc[n])   
            ref_waters.append(df.loc[n])

            break
print(len(ref_waters))

Water ID = 19.00
Water ID = 20.00
Water ID = 48.00
Water ID = 49.00
Water ID = 50.00
Water ID = 51.00
Water ID = 52.00
Water ID = 53.00
Water ID = 71.00
Water ID = 72.00
Water ID = 73.00
Water ID = 74.00
Water ID = 75.00
Water ID = 112.00
Water ID = 113.00
Water ID = 114.00
Water ID = 115.00
Water ID = 289.00
Water ID = 1684.00
19


# Make list of peaks in channel 

In [34]:
peaks=[]
for z,ruw in peaks_in_roi.iterrows():
    point1= [float(peaks_in_roi.loc[z,'x']),float(peaks_in_roi.loc[z,'y']),float(peaks_in_roi.loc[z,'z'])]
    for k,pew in  reference_midpt.iterrows():
        channel_roi = allrois[k]
        point2= [float(reference_midpt.loc[k,'x']),float(reference_midpt.loc[k,'y']),float(reference_midpt.loc[k,'z'])]
        if ((distance(point1,point2) < channel_roi) ):
                print ("PEAK ID = %0.2f" %peaks_in_roi['resseq'].loc[z])   
                peaks.append(peaks_in_roi.loc[z])
                break
print(len(peaks))

PEAK ID = 47.00
PEAK ID = 145.00
PEAK ID = 212.00
PEAK ID = 250.00
PEAK ID = 316.00
PEAK ID = 487.00
PEAK ID = 514.00
PEAK ID = 562.00
PEAK ID = 838.00
PEAK ID = 931.00
PEAK ID = 1058.00
PEAK ID = 1260.00
PEAK ID = 1410.00
PEAK ID = 1701.00
PEAK ID = 2117.00
PEAK ID = 2496.00
PEAK ID = 2722.00
PEAK ID = 3097.00
PEAK ID = 3983.00
PEAK ID = 7052.00
20


## Produce matches_all.txt files

In [35]:
matches= pd.DataFrame(columns=['XTAL ID', 'Chain' ,'Closest MD Partner', 'Dist', 'Thresh'],dtype=object) #xtal waters
matches_all =pd.DataFrame(columns=['MD ID', 'Closest XTAL Partner', 'Dist', "Thresh"],dtype=object)  #md_peaks
dist_array=[]
partner_array=[]
strength_array=[]
chain_array=[]
peaks_df =pd.DataFrame(peaks)
ref_waters_df =pd.DataFrame( ref_waters )

peaks_array=peaks_df['resseq']
water_array=ref_waters_df ['resseq']

for h,rhu in peaks_df.iterrows():
    point1= [float(peaks_df.loc[h,'x']),float(peaks_df.loc[h,'y']),float(peaks_df.loc[h,'z'])]
    dist=10000
    partner=0
    strength= peaks_df.loc[h,'occupancy']

    for z,r in ref_waters_df.iterrows():
        point2= [float(ref_waters_df.loc[z,'x']),float(ref_waters_df.loc[z,'y']),float(ref_waters_df.loc[z,'z'])]
        dist_try=distance(point1,point2)
        if dist_try <dist:
            dist=dist_try #update value
            partner= ref_waters_df.loc[z,'resseq']
            chain = ref_waters_df.loc[z,'chainid']
    dist_array.append(dist)
    partner_array.append(partner)      
    strength_array.append(strength)
    chain_array.append(chain)
matches_all['MD ID'] = peaks_array
matches_all['Closest XTAL Partner'] = partner_array
matches_all['Dist'] = dist_array
matches_all['Thresh'] = strength_array
matches_all['Chain'] = chain_array
print(np.mean(strength_array))
print(np.std(strength_array))


7.4220000000000015
1.8443280619239086


In [36]:
matches_all.to_pickle('files/Matches_XTALwater_%s_chain1' %chan)